In [4]:
import os, sys, torch, json
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# 1. 경로 설정 (반드시 확인!)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if ROOT_DIR not in sys.path: sys.path.append(ROOT_DIR)

from multilayer_model.m3_multi_bce import M3_Multi_BCE
from multilayer_model.m3_multi_mse import M3_Multi_MSE
from multilayer_loss.l_multi3_final_logic import l_multi3_final_loss
from vae_earlystopping import EarlyStopping

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Environment Ready. Device: {device}")

Environment Ready. Device: cuda


In [ ]:
# 2. 데이터 로드 및 전처리
x1_raw = np.load('../data/metal.npy')
x2_raw = np.load('../data/support_norm.npy')
x3_raw = np.load('../data/pre_fin.npy')
c_raw = np.load('../data/re_fin.npy')

tr_idx, te_idx = train_test_split(np.arange(len(x1_raw)), test_size=0.1, random_state=42)
tr_idx, va_idx = train_test_split(tr_idx, test_size=0.1, random_state=42)

sc1, sc2, sc3, scc = MinMaxScaler(), MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
def prep(d, i, s, fit=False): return s.fit_transform(d[i]) if fit else s.transform(d[i])

x1_tr, x1_va, x1_te = prep(x1_raw, tr_idx, sc1, True), prep(x1_raw, va_idx, sc1), prep(x1_raw, te_idx, sc1)
x2_tr, x2_va, x2_te = prep(x2_raw, tr_idx, sc2, True), prep(x2_raw, va_idx, sc2), prep(x2_raw, te_idx, sc2)
x3_tr, x3_va, x3_te = prep(x3_raw, tr_idx, sc3, True), prep(x3_raw, va_idx, sc3), prep(x3_raw, te_idx, sc3)
c_tr, c_va, c_te = prep(c_raw, tr_idx, scc, True), prep(c_raw, va_idx, scc), prep(c_raw, te_idx, scc)

def to_t(a): return torch.tensor(a, dtype=torch.float32)
train_loader = DataLoader(TensorDataset(to_t(x1_tr), to_t(x2_tr), to_t(x3_tr), to_t(c_tr)), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(to_t(x1_va), to_t(x2_va), to_t(x3_va), to_t(c_va)), batch_size=64)
test_loader = DataLoader(TensorDataset(to_t(x1_te), to_t(x2_te), to_t(x3_te), to_t(c_te)), batch_size=64)

x_dims = [x1_tr.shape[1], x2_tr.shape[1], x3_tr.shape[1]]
c_dim = c_tr.shape[1]

FileNotFoundError: [Errno 2] No such file or directory: '../data/pre_fin_norm.npy'

In [ ]:
# 3. 독립 모델 학습 함수
def train_expert(model, mode='mse'):
    opt = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
    es = EarlyStopping(patience=50)
    for epoch in range(1, 801):
        model.train()
        for b1, b2, b3, bc in train_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            opt.zero_grad()
            p, m, v = model(b1, b2, b3, bc)
            # KL 가중치를 낮춰 R2 향상 유도
            loss = l_multi3_final_loss(p, [b1, b2, b3], m, v, mode=mode, gamma_list=[0.01, 0.005, 0.001])
            loss.backward(); opt.step()
        
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v1, v2, v3, vc in val_loader:
                v1, v2, v3, vc = v1.to(device), v2.to(device), v3.to(device), vc.to(device)
                vp, vm, vv = model(v1, v2, v3, vc)
                v_loss += l_multi3_final_loss(vp, [v1, v2, v3], vm, vv, mode=mode).item()
        
        if epoch % 100 == 0: print(f"[{mode.upper()}] Epoch {epoch} | Val Loss: {v_loss/len(val_loader):.4f}")
        if es(v_loss/len(val_loader), model): break
    es.load_best_model(model)
    return model

print("Training Stage 1: BCE Specialist...")
m_bce = train_expert(M3_Multi_BCE(x_dims, c_dim).to(device), 'bce')
print("Training Stage 2: MSE Specialist...")
m_mse = train_expert(M3_Multi_MSE(x_dims, c_dim).to(device), 'mse')

In [ ]:
# 4. Pure Generation 기반 검증 (R2 극대화 로직)
m_bce.eval(); m_mse.eval()
preds, trues = [], []
with torch.no_grad():
    for b1, b2, b3, bc in test_loader:
        bc = bc.to(device)
        # 정답(b1,b2,b3) 없이 오직 조건(bc)으로만 생성!
        b_gen = m_bce.generate(bc, device)
        m_gen = m_mse.generate(bc, device)
        
        # Gating 적용 (마지막 단계: 전처리)
        prob3 = torch.sigmoid(b_gen[2])
        val3 = m_gen[2]
        combined = (prob3 * val3).cpu().numpy()
        
        preds.append(sc3.inverse_transform(combined))
        trues.append(sc3.inverse_transform(b3.cpu().numpy()))

y_p, y_t = np.concatenate(preds).flatten(), np.concatenate(trues).flatten()
print(f"\n🏆 Final Generation R2 (Pretreatment): {r2_score(y_t, y_p):.4f}")

plt.figure(figsize=(8, 5))
plt.scatter(y_t, y_p, alpha=0.5, color='orange')
plt.plot([y_t.min(), y_t.max()], [y_t.min(), y_t.max()], 'r--')
plt.title("Pure Generation Test (Pretreatment)")
plt.show()